# 发送推理请求

当我们将模型部署到服务端后，我们需要向服务端发送特定的请求以实现推理。因此，本节我们将编写一个请求发送脚本实现向Flask发送推理请求，并获取服务端返回的推理结果。我们将沿用本地部署的数据预处理模块，将输入的图像转换为符合模型推理要求的tensor。

根据上节服务端的设置，我们需要：
* 提取tensor的尺寸并转换为JSON数据
* 将tensor转换为字节流类型变量（bytearray）

随后，我们使用requests库将数据进行打包并发送请求。当服务端完成推理，推理结果将返回到变量r中。

**注：在Colab环境下，由于我们不能访问本地链接，因此服务端的部署地址将通过flask_ngrok映射到特定的链接中，如：**
<br/>Running on http://8af5-104-196-56-36.ngrok.io

**由于该链接会发生改变，请同学们在具体下述代码时根据之前启动Flask服务返回的结果，自行替换ngrok_link变量中的链接。**

In [ ]:
# Download Pretrained Model
!gdown --id 1omW_gpSpedWjo5GahxzksNW-J8MGrfdk --output convnet.pth

# Download Testing Image
!gdown --id 1A47zfadA6oXGhMvI-iCRj6CWB7rS8dWz --output digit_image.jpg

In [16]:
ngrok_link = 'http://cbf4-104-196-56-36.ngrok.io' # 请同学们自行替换对应的链接

In [ ]:
import io
import json
import requests
from PIL import Image

from torchvision import transforms

In [ ]:
image = Image.open("./digit_image.jpg")

In [ ]:
def image_to_tensor(image):
    gray_image = transforms.functional.to_grayscale(image)
    resized_image = transforms.functional.resize(gray_image, (28, 28))
    input_image_tensor = transforms.functional.to_tensor(resized_image)
    input_image_tensor_norm = transforms.functional.normalize(input_image_tensor, (0.1302,), (0.3069,))
    return input_image_tensor_norm

image_tensor = image_to_tensor(image)

In [ ]:
dimensions = json.dumps({'dims': list(image_tensor.shape)})
data = bytearray(image_tensor.numpy())

In [ ]:
dimensions

In [ ]:
data

In [17]:
r = requests.post(ngrok_link+'/test',
                  files={'metadata': dimensions, 'data' : data})

response = json.loads(r.content)

print("Predicted digit :", response)

Predicted digit : 2
